In [ ]:
!pip install wordfreq

In [ ]:
!apt-get install -y imagemagick

In [3]:
import os
import random
import numpy as np
import cv2
import pandas as pd
from pathlib import Path
from subprocess import run

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
import pandas as pd
df = pd.DataFrame(columns=["code", "word","language" ,"text_color","background_color", "font_size", "font_family", "font_style", "noise","angle","clean_path","noise_path","binary_mask_path"])

In [27]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import random
import io
import os

def to_pil(cv2_img):
    return Image.fromarray(cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB))

def to_cv2(pil_img):
    return cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)

def apply_noise(image, noise_type):
    if image is None:
        print(f"Warning: Could not read image. Skipping noise type: {noise_type}")
        return None

    pil_img = to_pil(image)
    cv2_img = image.copy()
    h, w = cv2_img.shape[:2]

    if noise_type == "blur":
        return cv2.GaussianBlur(cv2_img, (5, 5), 0)

    elif noise_type == "motion blur":
        kernel = np.zeros((15, 15))
        kernel[7, :] = np.ones(15)
        kernel /= 15
        return cv2.filter2D(cv2_img, -1, kernel)

    elif noise_type == "pixelation":
        h, w = cv2_img.shape[:2]
        temp = cv2.resize(cv2_img, (w//10, h//10), interpolation=cv2.INTER_LINEAR)
        return cv2.resize(temp, (w, h), interpolation=cv2.INTER_NEAREST)

    elif noise_type == "compression artifacts":
        encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 10]
        _, enc_img = cv2.imencode('.jpg', cv2_img, encode_param)
        return cv2.imdecode(enc_img, 1)

    elif noise_type == "Gaussian noise":
        row, col, ch = cv2_img.shape
        mean = 0
        sigma = 25
        gauss = np.random.normal(mean, sigma, (row, col, ch))
        gauss = np.clip(gauss, -sigma*3, sigma*3)
        noisy_img = cv2_img.astype(np.float32) + gauss
        noisy_img = np.clip(noisy_img, 0, 255).astype(np.uint8)
        return noisy_img

    elif noise_type == "low resolution":
        small = cv2.resize(cv2_img, (32, 32), interpolation=cv2.INTER_LINEAR)
        return cv2.resize(small, (cv2_img.shape[1], cv2_img.shape[0]), interpolation=cv2.INTER_NEAREST)

    elif noise_type == "image stretching":
        h, w = cv2_img.shape[:2]
        return cv2.resize(cv2_img, (int(w * 1.5), h))

    elif noise_type == "defocus blur":
        return cv2.GaussianBlur(cv2_img, (21, 21), 0)

    elif noise_type == "ringing artifacts":
        blurred = cv2.GaussianBlur(cv2_img, (0, 0), 3)
        return cv2.addWeighted(cv2_img, 1.5, blurred, -0.5, 0)

    elif noise_type == "lighting variations":
        enhancer = ImageEnhance.Brightness(pil_img)
        return to_cv2(enhancer.enhance(random.uniform(0.5, 1.5)))

    elif noise_type == "uneven illumination":
        h, w = cv2_img.shape[:2]
        mask = np.zeros((h, w), np.uint8)
        cv2.circle(mask, (w//2, h//2), w//2, 255, -1)
        mask_3_channel = cv2.merge([mask]*3)
        return cv2.addWeighted(cv2_img, 1, mask_3_channel, 0.3, 0)

    elif noise_type == "shadows":
        mask = np.zeros_like(cv2_img)
        x1 = random.randint(0, w-1)
        y1 = random.randint(0, h-1)
        x2 = random.randint(0, w-1)
        y2 = random.randint(0, h-1)
        cv2.rectangle(mask, (min(x1, x2), min(y1, y2)), (max(x1, x2), max(y1, y2)), (0, 0, 0), -1)
        return cv2.addWeighted(cv2_img, 1, mask, 0.5, 0)

    elif noise_type == "reflections":
        overlay = np.full_like(cv2_img, 255)
        beta_weight = 0.3
        return cv2.addWeighted(cv2_img, 1 - beta_weight, overlay, beta_weight, 0)

    elif noise_type == "glare":
        glare = np.zeros_like(cv2_img)
        center = (random.randint(0, w), random.randint(0, h))
        cv2.circle(glare, center, random.randint(50, 100), (255, 255, 255), -1)
        return cv2.addWeighted(cv2_img, 0.7, glare, 0.3, 0)

    elif noise_type == "overexposure":
        return cv2.convertScaleAbs(cv2_img, alpha=1.5, beta=50)

    elif noise_type == "underexposure":
        return cv2.convertScaleAbs(cv2_img, alpha=0.5, beta=-50)

    elif noise_type == "background clutter":
        noise_bg = np.random.randint(0, 255, cv2_img.shape, dtype='uint8')
        return cv2.addWeighted(cv2_img, 0.8, noise_bg, 0.2, 0)

    elif noise_type == "ambient noise":
        ambient = cv2.GaussianBlur(cv2_img, (9, 9), 10)
        return cv2.addWeighted(cv2_img, 0.7, ambient, 0.3, 0)

    elif noise_type == "perspective distortion":
        pts1 = np.float32([[0,0], [w,0], [0,h], [w,h]])
        pts2 = np.float32([[random.randint(0, w//10), random.randint(0, h//10)],
                           [w-random.randint(0, w//10), random.randint(0, h//10)],
                           [random.randint(0, w//10), h-random.randint(0, h//10)],
                           [w-random.randint(0, w//10), h-random.randint(0, h//10)]])
        M = cv2.getPerspectiveTransform(pts1, pts2)
        return cv2.warpPerspective(cv2_img, M, (w, h))

    elif noise_type == "fading":
        alpha = np.linspace(1, 0.3, w)
        mask = np.tile(alpha, (h, 1))
        return (cv2_img * mask[:, :, np.newaxis]).astype(np.uint8)

    elif noise_type == "ink bleed-through":
        flipped = cv2.flip(cv2_img, 1)
        blended = cv2.addWeighted(cv2_img, 0.8, flipped, 0.2, 0)
        return blended

    elif noise_type == "text smearing":
        kernel = np.ones((1, 9), np.uint8)
        return cv2.dilate(cv2_img, kernel, iterations=1)

    elif noise_type == "stroke breaks":
        mask = np.random.randint(0, 2, size=(h, w, 1), dtype=np.uint8)
        mask_3_channel = cv2.merge([mask[:,:,0]]*3)
        return cv2_img * mask_3_channel

    elif noise_type == "partial occlusion":
        noisy_img = cv2_img.copy()
        for _ in range(3):
            x1, y1 = random.randint(0, max(0, w-50)), random.randint(0, max(0, h-50))
            x2, y2 = x1 + random.randint(20, 50), y1 + random.randint(20, 50)
            cv2.rectangle(noisy_img, (x1, y1), (x2, y2), (0, 0, 0), -1)
        return noisy_img

    elif noise_type == "font erosion":
        kernel = np.ones((2, 2), np.uint8)
        return cv2.erode(cv2_img, kernel, iterations=1)

    elif noise_type == "character merging":
        kernel = np.ones((2, 2), np.uint8)
        return cv2.dilate(cv2_img, kernel, iterations=2)

    elif noise_type == "JPEG compression":
        encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 5]
        _, enc_img = cv2.imencode('.jpg', cv2_img, encode_param)
        decoded_img = cv2.imdecode(enc_img, cv2.IMREAD_COLOR)
        return decoded_img if decoded_img is not None else cv2_img

    elif noise_type == "color space mismatch":
        return cv2.cvtColor(cv2_img, cv2.COLOR_BGR2HSV)

    elif noise_type == "resolution scaling":
        small = cv2.resize(cv2_img, (w//2, h//2), interpolation=cv2.INTER_LINEAR)
        return cv2.resize(small, (w, h), interpolation=cv2.INTER_NEAREST)

    elif noise_type == "quantization noise":
        levels = 16
        return (cv2_img // levels * levels).astype(np.uint8)

    elif noise_type == "aliasing":
        small = cv2.resize(cv2_img, (w//4, h//4), interpolation=cv2.INTER_LINEAR)
        return cv2.resize(small, (w, h), interpolation=cv2.INTER_NEAREST)

    elif noise_type == "banding":
        banded = cv2_img.copy()
        for i in range(0, h, 20):
            start_row = i
            end_row = min(i + 5, h)
            banded[start_row:end_row] = banded[start_row:end_row] // 2
        return banded

    elif noise_type == "bit-depth reduction":
        return (cv2_img >> 3) << 3

    elif noise_type == "encoding artifacts":
        temp = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2GRAY)
        _, binary = cv2.threshold(temp, 127, 255, cv2.THRESH_BINARY)
        return cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)

    elif noise_type == "scan line artifacts":
        scan_img = cv2_img.copy()
        for i in range(0, h, 5):
             start_row = i
             end_row = min(i + 1, h)
             scan_img[start_row:end_row] = scan_img[start_row:end_row] // 3
        return scan_img

    elif noise_type == "scanner streaks":
        streak = cv2_img.copy()
        for _ in range(5):
            x = random.randint(0, w-1)
            streak[:, x] = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
        return streak

    elif noise_type == "printer toner gaps":
        toner = cv2_img.copy()
        for _ in range(3):
            y = random.randint(0, max(0, h-20)) # Ensure y is valid
            toner[y:min(y+5, h), :] = [255, 255, 255] # White gaps
        return toner

    elif noise_type == "misaligned scanning":
        tx = random.randint(-10, 10)
        ty = random.randint(-5, 5)
        M = np.float32([[1, 0, tx], [0, 1, ty]])
        return cv2.warpAffine(cv2_img, M, (w, h))

    elif noise_type == "fold marks":
        folded = cv2_img.copy()
        for _ in range(2):
            y = random.randint(0, h-1)
            cv2.line(folded, (0, y), (w, y), (200, 200, 200), 2)
        return folded

    elif noise_type == "paper texture interference":
        noise = np.random.normal(127, 25, cv2_img.shape).astype(np.uint8)
        return cv2.addWeighted(cv2_img, 0.9, noise, 0.1, 0)

    elif noise_type == "multi-layer noise":
        noisy = apply_noise(cv2_img, "Gaussian noise")
        if noisy is not None:
          noisy = apply_noise(noisy, "text smearing")
        if noisy is not None:
          return apply_noise(noisy, "JPEG compression")
        return cv2_img

    elif noise_type == "watermarks":
        watermark = cv2.putText(cv2_img.copy(), "WATERMARK", (w//4, h//2),
                                cv2.FONT_HERSHEY_SIMPLEX, min(1.5, h/50), (150, 150, 150), max(1, w//200), cv2.LINE_AA) # Adjust font scale/thickness based on image size
        return cv2.addWeighted(cv2_img, 0.85, watermark, 0.15, 0)

    elif noise_type == "overlay interference":
        overlay = np.full_like(cv2_img, 100)
        return cv2.addWeighted(cv2_img, 0.75, overlay, 0.25, 0)

    if noise_type == "salt and pepper":
        prob = 0.01
        noisy = cv2_img.copy()
        black = np.random.rand(h, w) < prob
        white = np.random.rand(h, w) < prob
        noisy[black] = 0
        noisy[white] = 255
        return noisy

    elif noise_type == "ink spread":
        kernel = np.ones((3, 3), np.uint8)
        return cv2.dilate(cv2_img, kernel, iterations=2)

    elif noise_type == "horizontal tear":
        y = random.randint(h // 3, 2 * h // 3)
        cv2.line(cv2_img, (0, y), (w, y), (255, 255, 255), thickness=5)
        return cv2_img

    elif noise_type == "vertical tear":
        x = random.randint(w // 3, 2 * w // 3)
        cv2.line(cv2_img, (x, 0), (x, h), (255, 255, 255), thickness=5)
        return cv2_img

    elif noise_type == "smudge":
        smudge_kernel = (9, 9)
        smudged = cv2.blur(cv2_img, smudge_kernel)
        return cv2.addWeighted(cv2_img, 0.6, smudged, 0.4, 0)

    elif noise_type == "text ghosting":
        shifted = np.roll(cv2_img, 5, axis=1)
        return cv2.addWeighted(cv2_img, 0.7, shifted, 0.3, 0)

    elif noise_type == "double scan":
        shifted = np.roll(cv2_img, random.randint(2, 6), axis=0)
        return cv2.addWeighted(cv2_img, 0.5, shifted, 0.5, 0)

    elif noise_type == "wave distortion":
        distorted = np.zeros_like(cv2_img)
        for i in range(h):
            offset = int(10.0 * np.sin(2 * np.pi * i / 60))
            distorted[i] = np.roll(cv2_img[i], offset, axis=1)
        return distorted

    elif noise_type == "chromatic aberration":
        b, g, r = cv2.split(cv2_img)
        b = np.roll(b, 1, axis=1)
        r = np.roll(r, -1, axis=1)
        return cv2.merge([b, g, r])

    elif noise_type == "paper curl":
        mask = np.zeros((h, w), dtype=np.uint8)
        cv2.rectangle(mask, (0, 0), (w, h), 255, thickness=30)
        darken = cv2.merge([mask // 3] * 3)
        return cv2.subtract(cv2_img, darken)

    else:
        return cv2_img

In [28]:
implemented_noise_types = [
    "blur", "motion blur", "pixelation", "compression artifacts", "Gaussian noise",
    "low resolution", "image stretching", "defocus blur", "ringing artifacts",
    "lighting variations", "uneven illumination", "shadows", "reflections",
    "glare", "overexposure", "underexposure", "background clutter", "ambient noise",
    "perspective distortion", "fading", "ink bleed-through", "text smearing",
    "stroke breaks", "partial occlusion", "font erosion", "character merging",
    "JPEG compression", "color space mismatch", "resolution scaling",
    "quantization noise", "aliasing", "banding", "bit-depth reduction",
    "encoding artifacts", "scan line artifacts", "scanner streaks",
    "printer toner gaps", "misaligned scanning", "fold marks",
    "paper texture interference", "multi-layer noise", "watermarks",
    "overlay interference",
    "salt and pepper", "ink spread", "horizontal tear", "vertical tear",
    "smudge", "text ghosting", "double scan", "wave distortion",
    "chromatic aberration", "paper curl"
]

In [29]:
def generate_mask(clean_img, noisy_img):
    # Resize noisy image to match clean image dimensions (if needed)
    if clean_img.shape != noisy_img.shape:
        noisy_img = cv2.resize(noisy_img, (clean_img.shape[1], clean_img.shape[0]))
        # If clean_img is color and noisy_img is grayscale, convert noisy_img to color
        if len(clean_img.shape) == 3 and len(noisy_img.shape) == 2:
            noisy_img = cv2.cvtColor(noisy_img, cv2.COLOR_GRAY2BGR)
        elif len(clean_img.shape) == 2 and len(noisy_img.shape) == 3:
            clean_img = cv2.cvtColor(clean_img, cv2.COLOR_GRAY2BGR)

    diff = cv2.absdiff(clean_img, noisy_img)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, 25, 255, cv2.THRESH_BINARY)
    return mask

In [30]:
import subprocess
import os
import random
from pathlib import Path
import cv2
import numpy as np
from PIL import Image
import shutil
output_root = '/content/drive/MyDrive/IITJ_Project/English_dataset'
def generate_text_image(
    text: str,
    font_path: str,
    font_size: int,
    output_path: str,
    text_color="black",
    background_color="white",
    rotate_prob=0.3,
    angle=(-30, 30)  # degrees
):
    command = [
        "convert",
        "-background", background_color,
        "-fill", text_color,
        "-font", str(font_path),
        "-pointsize", str(font_size),
        f"label:{text}",
        "-trim",                   # remove extra white space
        "+repage",                 # reset virtual canvas after trim
        "-rotate", str(angle),
        output_path
    ]

    try:
        subprocess.run(command, check=True)
        print(f"Saved clean image: {output_path}")
    except subprocess.CalledProcessError as e:
        print("Failed to generate image:", e)

def generate_and_apply_noise(
    words,
    fonts_dir,
    clean_output_dir,
    noisy_output_dir,
    bitmask_output_dir,
    num_samples=100,
    text_colors = [
    "black", "darkblue", "darkgreen", "darkred",
    "navy", "maroon", "teal", "purple", "indigo",
    "darkslategray", "saddlebrown"
    ],
    background_colors = [
    "white",           # baseline
    "lightgrey",       # baseline
    "lightyellow",     # soft but warmer than ivory
    "lavender",        # soft bluish-purple
    "aliceblue",       # pale blue
    "beige",           # subtle brown-tinted
    "seashell",        # very light pinkish tone
    "whitesmoke",      # smoky white, more greyish
    "gainsboro"        # slightly darker than lightgrey
    ],
    font_size_range=(32, 128),
    implemented_noise_types=implemented_noise_types,
    rotate_prob=0.3,
    rotate_range=(-100, 100)
):
    if implemented_noise_types is None:
        implemented_noise_types = [
            "Gaussian noise", "blur", "motion blur", "pixelation"
            # Add your full list here or more implementations in apply_noise()
        ]

    clean_output_dir = Path(clean_output_dir)
    noisy_output_dir = Path(noisy_output_dir)
    bitmask_output_dir = Path(bitmask_output_dir)
    clean_output_dir.mkdir(parents=True, exist_ok=True)
    noisy_output_dir.mkdir(parents=True, exist_ok=True)
    bitmask_output_dir.mkdir(parents=True,exist_ok=True)

    font_families = [f for f in Path(fonts_dir).iterdir() if f.is_dir()]
    if not font_families:
        raise ValueError("No font family folders found.")

    image_counter = 0

    for word in words:
        angle = 0
        if random.random() <= rotate_prob:
          angle = int(round(random.uniform(*rotate_range)))

        random_family = random.choice(font_families)

        fonts = list(random_family.glob("*.ttf")) + list(random_family.glob("*.otf"))
        if not fonts:
            print(f"No fonts found in {random_family}, skipping...")
            continue

        random_font = random.choice(fonts)
        color = random.choice(text_colors)
        background_color=random.choice(background_colors)
        font_name = Path(random_font).stem.replace(" ", "_")
        font_size = random.randint(*font_size_range)


        # File naming for clean image
        clean_filename = f"{image_counter:04d}_{random_family.name}_{font_name}_{color}_{font_size}.png"
        clean_path = clean_output_dir / clean_filename

        # Generate clean synthetic text image
        generate_text_image(
            text=word,
            font_path=random_font,
            font_size=font_size,
            output_path=str(clean_path),
            text_color=color,
            background_color=background_color,
            rotate_prob=0.3,
            angle=angle
        )

        # Load clean image with OpenCV
        image = cv2.imread(str(clean_path))
        if image is None:
            print(f"Failed to read image {clean_path}, skipping noise application.")
            continue

        # Choose noise type and apply noise
        noise_type = random.choice(implemented_noise_types)
        noisy_image = apply_noise(image, noise_type)

        # Save noisy image with noise type appended in filename
        noisy_filename = f"{clean_path.stem}_{noise_type.replace(' ', '_')}.jpg"
        noisy_path = noisy_output_dir / noisy_filename


        noisy_img = cv2.imread(str(noisy_path))
        cv2.imwrite(str(noisy_path), noisy_image)
        print(f"Saved noisy image: {noisy_path}")
        # Bitmask filename and path
        bitmask_filename = f"{clean_path.stem}_{noise_type.replace(' ', '_')}_mask.png"
        bitmask_path = bitmask_output_dir / bitmask_filename


        bit_mask_img = generate_mask(image, noisy_image)
        cv2.imwrite(str(bitmask_path), bit_mask_img)
        print(f"Saved bitmask image: {bitmask_path}")

        df.loc[len(df)] = [
            f"{image_counter:04d}",  # code (id)
            word,                   # word (text)
            "english",
            color,                  # text_color
            background_color,
            font_size,              # font_size
            random_family.name,       # font_family
            font_name,              # font_style (font file name)
            noise_type,              # noise applied
            angle,
            clean_path,
            noisy_path,
            bitmask_path
        ]

        image_counter += 1



In [31]:
from wordfreq import top_n_list

all_words = top_n_list('en', 10000)

filtered_words = [word for word in all_words if len(word) > 4]

final_words = filtered_words[:100]

print(len(final_words))
print(final_words)

100
['about', 'their', 'there', 'which', 'would', 'people', "don't", 'other', 'after', 'first', 'think', 'could', 'these', 'because', 'where', 'should', 'really', 'right', 'years', 'being', 'going', 'before', 'still', 'never', 'those', 'world', 'great', 'through', "you're", "that's", 'while', 'something', "can't", 'every', 'state', 'three', 'around', 'between', 'always', 'better', 'little', 'since', 'another', 'things', 'under', 'during', 'thing', 'house', 'place', 'school', 'again', 'without', 'against', "didn't", 'found', 'family', 'might', 'please', 'money', 'second', 'someone', 'number', 'night', 'until', 'company', 'doing', 'called', 'different', 'having', 'thought', 'however', 'getting', 'government', 'group', 'looking', 'public', 'women', 'business', 'start', 'system', 'times', 'already', 'anything', 'nothing', 'person', 'today', 'change', 'enough', 'everything', 'making', 'point', "there's", "doesn't", 'support', 'including', 'music', 'power', 'states', 'water', 'based']


In [32]:
# Set output root folder (Google Drive)
output_root = '/content/drive/MyDrive/IITJ_Project/English_dataset'

# Create the three folders inside output_root
clean_dir = os.path.join(output_root, 'clean')
noisy_dir = os.path.join(output_root, 'noisy')
bitmask_dir = os.path.join(output_root, 'bitmask')

os.makedirs(clean_dir, exist_ok=True)
os.makedirs(noisy_dir, exist_ok=True)
os.makedirs(bitmask_dir, exist_ok=True)
generate_and_apply_noise(
    words=final_words,
    fonts_dir="/content/drive/MyDrive/IITJ_Project/English_dataset/font-family",  # your fonts folder
    clean_output_dir=clean_dir,  # output clean images
    noisy_output_dir=noisy_dir,  # output noisy images
    bitmask_output_dir=bitmask_dir,
    num_samples=100,
    text_colors=["black", "blue", "red", "green", "purple"],
    font_size_range=(40, 120),
    rotate_prob=0.3,
    rotate_range=(-15, 15)
)

metadata_csv_path = os.path.join(output_root, "metadata.csv")
df = pd.DataFrame(df)
df.to_csv(metadata_csv_path, index=False)

Saved clean image: /content/drive/MyDrive/IITJ_Project/English_dataset/clean/0000_Roboto_Roboto-ExtraBold_red_91.png
Saved noisy image: /content/drive/MyDrive/IITJ_Project/English_dataset/noisy/0000_Roboto_Roboto-ExtraBold_red_91_glare.jpg
Saved bitmask image: /content/drive/MyDrive/IITJ_Project/English_dataset/bitmask/0000_Roboto_Roboto-ExtraBold_red_91_glare_mask.png
Saved clean image: /content/drive/MyDrive/IITJ_Project/English_dataset/clean/0001_Roboto_Roboto-ExtraLight_green_46.png
Saved noisy image: /content/drive/MyDrive/IITJ_Project/English_dataset/noisy/0001_Roboto_Roboto-ExtraLight_green_46_image_stretching.jpg
Saved bitmask image: /content/drive/MyDrive/IITJ_Project/English_dataset/bitmask/0001_Roboto_Roboto-ExtraLight_green_46_image_stretching_mask.png
Saved clean image: /content/drive/MyDrive/IITJ_Project/English_dataset/clean/0002_Arvo_Arvo-Bold_green_84.png
Saved noisy image: /content/drive/MyDrive/IITJ_Project/English_dataset/noisy/0002_Arvo_Arvo-Bold_green_84_perspecti

In [33]:
df

,code,word,language,text_color,background_color,font_size,font_family,font_style,noise,angle,clean_path,noise_path,binary_mask_path
0,0000,about,english,red,lightgrey,91,Roboto,Roboto-ExtraBold,glare,0,/content/drive/MyDrive/IITJ_Project/English_da...,/content/drive/MyDrive/IITJ_Project/English_da...,/content/drive/MyDrive/IITJ_Project/English_da...
1,0001,their,english,green,lightyellow,46,Roboto,Roboto-ExtraLight,image stretching,-7,/content/drive/MyDrive/IITJ_Project/English_da...,/content/drive/MyDrive/IITJ_Project/English_da...,/content/drive/MyDrive/IITJ_Project/English_da...
2,0002,there,english,green,lightyellow,84,Arvo,Arvo-Bold,perspective distortion,0,/content/drive/MyDrive/IITJ_Project/English_da...,/content/drive/MyDrive/IITJ_Project/English_da...,/content/drive/MyDrive/IITJ_Project/English_da...
3,0003,which,english,red,seashell,69,Great_Vibes,GreatVibes-Regular,watermarks,0,/content/drive/MyDrive/IITJ_Project/English_da...,/content/drive/MyDrive/IITJ_Project/English_da...,/content/drive/MyDrive/IITJ_Project/English_da...
4,0004,would,english,purple,gainsboro,78,Nunito,Nunito-Black,scanner streaks,0,/content/drive/MyDrive/IITJ_Project/English_da...,/content/drive/MyDrive/IITJ_Project/English_da...,/content/drive/MyDrive/IITJ_Project/English_da...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0095,music,english,green,aliceblue,45,Playfair_Display,Copy_of_PlayfairDisplay-MediumItalic,perspective distortion,14,/content/drive/MyDrive/IITJ_Project/English_da...,/content/drive/MyDrive/IITJ_Project/English_da...,/content/drive/MyDrive/IITJ_Project/English_da...
96,0096,power,english,black,whitesmoke,58,Lora,Lora-Bold,banding,0,/content/drive/MyDrive/IITJ_Project/English_da...,/content/drive/MyDrive/IITJ_Project/English_da...,/content/drive/MyDrive/IITJ_Project/English_da...
97,0097,states,english,red,lightyellow,74,Pacifico,Pacifico-Regular,overlay interference,0,/content/drive/MyDrive/IITJ_Project/English_da...,/content/drive/MyDrive/IITJ_Project/English_da...,/content/drive/MyDrive/IITJ_Project/English_da...
98,0098,water,english,black,lavender,82,Roboto,"Roboto-VariableFont_wdth,wght",partial occlusion,0,/content/drive/MyDrive/IITJ_Project/English_da...,/content/drive/MyDrive/IITJ_Project/English_da...,/content/drive/MyDrive/IITJ_Project/English_da...


In [ ]:
df.to_csv('metadata.csv', index=False)